In [3]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Liberation Serif'

import logging
import pickle
from datetime import datetime
import os
from data_processing import gen_ckpt_steps
from conv_plots_funcs import get_seg_starts_per_config
import torch
import gc
from core.config import Config
from data_train import set_config_params, gen_ckpt_pred_steps
from get_last_checkpoint import split_path
from haystack_plots import load_quartiles_ckpt_files
from models import GPT2

/home/sultand/TFs_do_KF_ICL/src
Using device: cuda
/home/sultand/TFs_do_KF_ICL/src
Using device: cuda
CUDA_VISIBLE_DEVICES: None


In [4]:
config = Config()
model_name = "ortho_haar_tiny" #"ortho_haar_medium_single_gpu"

output_dir, ckpt_dir, experiment_name = set_config_params(config, model_name)

datasource = "val"


colors = ['#000000', '#005CAB', '#E31B23', '#FFC325', '#00A651', '#9B59B6']



ORTHO HAAR TINY MODEL




In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"  # check if cuda is available

ckpt = 1000
filename = f"step={ckpt}.ckpt"
ckpt_path = ckpt_dir + "/checkpoints/" + filename

config.override("ckpt_path", ckpt_path)
#load model from ckpt_dir``
model = GPT2.load_from_checkpoint(config.ckpt_path,
                                n_dims_in=config.n_dims_in, n_positions=config.n_positions,
                                n_dims_out=config.n_dims_out, n_embd=config.n_embd,
                                n_layer=config.n_layer, n_head=config.n_head, use_pos_emb=config.use_pos_emb, map_location=device, strict=True).eval().to(device)

print(model)

GPT2(
  (_read_in): Linear(in_features=57, out_features=72, bias=True)
  (_backbone): GPT2Model(
    (wte): Embedding(50257, 72)
    (wpe): Embedding(2048, 72)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-2): 3 x GPT2Block(
        (ln_1): LayerNorm((72,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=216, nx=72)
          (c_proj): Conv1D(nf=72, nx=72)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((72,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=288, nx=72)
          (c_proj): Conv1D(nf=72, nx=288)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((72,), eps=1e-05, elementwise_affine=True)
  )
  (_read_out): Linear(in_features=72, out_features=5, bias=True)
)


In [12]:
total_params = 0
for name, param in model.named_parameters():
    if not ("wte" in name or "wpe" in name):
        # print(f"{name}: {param.shape}, {param.numel()}")
        total_params += param.numel()
print(f"Total parameters (excluding wte and wpe): {total_params:,}")

Total parameters (excluding wte and wpe): 194,117


In [16]:
def parameter_count(model, context_len):
    total_params = 0
    for name, param in model.named_parameters():
        if not ("wte" in name or "wpe" in name):
            total_params += param.numel()

        #get the shape of wpe
        elif "wpe" in name:
            wpe_shape = param.shape
            print(f"{name}: {param.shape}, {param.numel()}")
            wpe_params = context_len * wpe_shape[1]
            total_params += wpe_params
            print(f"wpe params: {wpe_params:,}")

    print(f"Total parameters (excluding wte and wpe): {total_params:,}")
    return total_params

In [17]:
param_count = parameter_count(model, config.n_positions+1)

_backbone.wpe.weight: torch.Size([2048, 72]), 147456
wpe params: 18,072
Total parameters (excluding wte and wpe): 212,189
